In [68]:
import pandas as pd
import argparse
from nilearn.glm.second_level import SecondLevelModel
from nilearn import plotting
import os
from nilearn.glm.second_level import make_second_level_design_matrix
from nilearn.maskers import NiftiMasker

group = "gangnam_sad"
variable = "LSAS"
smoothness = 6
mdmr_dir = os.path.expanduser("~/fmri_project/C-PAC/CPAC/bcb_mdmr/")
nas_dir = os.path.expanduser("~/fmri_project/C-PAC/CPAC/bcb_mdmr/output/")
MDMR_output_dir = f"{nas_dir}/SAD_gangnam_MDMR/"
fmri_prep_dir = f"{nas_dir}/SAD_gangnam_resting_2/fMRIPrep_total"
seed_anal_dir = f"{nas_dir}/SAD_gangnam_seed_based_analysis/"

regressor_df = pd.read_csv(
    f"{mdmr_dir}/input/{group}_{variable}_regressor.csv"
)

subjects_label = regressor_df["Participant"].values
# 필요한 열만 선택하여 디자인 매트릭스 생성

extra_info_subjects = pd.DataFrame({
    "subject_label": subjects_label,
    "LSAS": regressor_df["LSAS"],
    "sex": regressor_df["SEX"],
    "age": regressor_df["AGE"],
    "yr_edu": regressor_df["YR_EDU"],
    "mean_framewise_displacement": regressor_df["Mean_Framewise_Displacement"]
})
design_matrix = make_second_level_design_matrix(
    subjects_label, extra_info_subjects
)


/home/brain/.conda/envs/gptpred/lib/python3.11/site-packages/nilearn/glm/first_level/design_matrix.py:508: UserWarning: Attention: Design matrix is singular. Aberrant estimates are expected.
  warn(


In [69]:
z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]

second_level_model = second_level_model.fit(
    z_maps,
    design_matrix=design_matrix,
)

In [70]:
z_map = second_level_model.compute_contrast("LSAS", output_type='z_score')

result_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/result/"
temp_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/temp/"
os.makedirs(result_dir, exist_ok=True)
from nilearn.glm import threshold_stats_img
z_map_filename = f"./unthresholded_z_map.nii.gz"
z_map.to_filename(z_map_filename)

thresholded_map2, threshold2 = threshold_stats_img(
    z_map, alpha=0.05, height_control="fdr"
)
print(f"The FDR=.05 threshold is {threshold2:.3g}")
thresholded_map2_filename = f"./FDR_correted_z_map.nii.gz"
thresholded_map2.to_filename(thresholded_map2_filename)

thresholded_map3, threshold3 = threshold_stats_img(
    z_map, alpha=0.05, height_control="bonferroni"
)
print(f"The p<.05 Bonferroni-corrected threshold is {threshold3:.3g}")
thresholded_map3_filename = f"./bonferroni_correted_z_map.nii.gz"
thresholded_map3.to_filename(thresholded_map3_filename)

The FDR=.05 threshold is 2.44
The p<.05 Bonferroni-corrected threshold is 4.71


In [71]:
p_map = second_level_model.compute_contrast(
    second_level_contrast = "LSAS", 
    output_type='p_value')

p_map_filename = f"./p_value_map.nii.gz"
p_map.to_filename(p_map_filename)

In [59]:
from nilearn.glm.second_level import non_parametric_inference


out_dict = non_parametric_inference(
    z_maps,
    design_matrix=design_matrix,
    second_level_contrast="LSAS",
    n_perm=15000,  # 500 for the sake of time. Ideally, this should be 10,000.
    two_sided_test=True,
    n_jobs=-1,
    threshold=0.005,
    verbose=1,
)

Fitting second level model...
Computation of second level model done in 0.17004847526550293 seconds
/home/brain/.conda/envs/gptpred/lib/python3.11/site-packages/nilearn/mass_univariate/permuted_least_squares.py:986: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  image.new_img_like(masker.mask_img_, metric_map),
/home/brain/.conda/envs/gptpred/lib/python3.11/site-packages/nilearn/masking.py:980: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(mask_img, unmasked, affine)


In [61]:
import numpy as np
import nibabel as nib

def threshold_results(out_dict, voxel_threshold=0.005, cluster_threshold=0.05):
    # Get images directly from out_dict
    logp_max_t_img = out_dict['logp_max_t']
    logp_max_size_img = out_dict['logp_max_size']

    # Get data and affine
    logp_max_t_data = logp_max_t_img.get_fdata()
    logp_max_size_data = logp_max_size_img.get_fdata()
    affine = logp_max_t_img.affine

    # Convert -log10(p) to p-values
    voxel_p_values = 10 ** (-logp_max_t_data)
    cluster_p_values = 10 ** (-logp_max_size_data)

    # Calculate -log10(p) thresholds
    voxel_logp_threshold = -np.log10(voxel_threshold)
    cluster_logp_threshold = -np.log10(cluster_threshold)

    # Create masks
    voxel_mask = logp_max_t_data > voxel_logp_threshold
    cluster_mask = logp_max_size_data > cluster_logp_threshold

    # Combine masks
    combined_mask = voxel_mask & cluster_mask

    # Count voxels in each mask
    total_voxels = np.prod(voxel_mask.shape)
    voxel_mask_count = np.sum(voxel_mask)
    cluster_mask_count = np.sum(cluster_mask)
    combined_mask_count = np.sum(combined_mask)

    # Print voxel counts and thresholds
    print(f"총 복셀 수: {total_voxels}")
    print(f"복셀 단위 임계값 (-log10(p) > {voxel_logp_threshold:.4f})을 통과한 복셀 수: {voxel_mask_count}")
    print(f"클러스터 단위 임계값 (-log10(p) > {cluster_logp_threshold:.4f})을 통과한 복셀 수: {cluster_mask_count}")
    print(f"두 임계값을 모두 통과한 복셀 수: {combined_mask_count}")

    # Apply combined mask to voxel-wise -log10(p) values
    final_logp_values = np.where(combined_mask, logp_max_t_data, 0)

    # Create and save the final images
    final_t_img = nib.Nifti1Image(logp_max_t_data, affine)
    final_size_img = nib.Nifti1Image(logp_max_size_data, affine)
    final_combined_img = nib.Nifti1Image(final_logp_values, affine)

    nib.save(final_t_img, 'logp_max_t.nii.gz')
    nib.save(final_size_img, 'logp_max_size.nii.gz')
    nib.save(final_combined_img, 'thresholded_logp_values.nii.gz')

    return final_combined_img

# Usage
thresholded_img = threshold_results(out_dict)

# Print -log10(p) values for p=0.005 and p=0.05
print(f"-log10(0.005) = {-np.log10(0.005):.4f}")
print(f"-log10(0.05) = {-np.log10(0.05):.4f}")

총 복셀 수: 84084
복셀 단위 임계값 (-log10(p) > 2.3010)을 통과한 복셀 수: 0
클러스터 단위 임계값 (-log10(p) > 1.3010)을 통과한 복셀 수: 2357
두 임계값을 모두 통과한 복셀 수: 0
-log10(0.005) = 2.3010
-log10(0.05) = 1.3010


In [ ]:
p_map_filename = "./logp_max_t.nii.gz"
out_dict["logp_max_t"].to_filename(p_map_filename)

p_map_filename = "./logp_max_size.nii.gz"
out_dict["logp_max_size"].to_filename(p_map_filename)

In [52]:
second_level_model = SecondLevelModel(n_jobs=-1)
z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]

second_level_model = second_level_model.fit(
    z_maps,
    design_matrix=design_matrix,
)

output = second_level_model.compute_contrast(
    second_level_contrast="LSAS",
    output_type="p_value"
)

p_map_filename = "./p_val.nii.gz"
output.to_filename(p_map_filename)

In [65]:
z_map = second_level_model.compute_contrast("LSAS", output_type='z_score')
from nilearn.glm import threshold_stats_img

thresholded_map2, threshold2 = threshold_stats_img(
    z_map, alpha=0.05, height_control="fdr", cluster_threshold = 0.05
)
print(f"The FDR=.005 threshold is {threshold2:.3g}")
thresholded_map2_filename = "./FDR_correted_z_map.nii.gz"
thresholded_map2.to_filename(thresholded_map2_filename)

thresholded_map3, threshold3 = threshold_stats_img(
    z_map, alpha=0.05, height_control="bonferroni"
)
print(f"The p<.05 Bonferroni-corrected threshold is {threshold3:.3g}")
thresholded_map3_filename = f"bonferroni_correted_z_map.nii.gz"
thresholded_map3.to_filename(thresholded_map3_filename)

The FDR=.005 threshold is 2.44
The p<.05 Bonferroni-corrected threshold is 4.71


Fitting second level model...
Computation of second level model done in 0.15488076210021973 seconds
/home/brain/.conda/envs/gptpred/lib/python3.11/site-packages/nilearn/mass_univariate/permuted_least_squares.py:986: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  image.new_img_like(masker.mask_img_, metric_map),
/home/brain/.conda/envs/gptpred/lib/python3.11/site-packages/nilearn/masking.py:980: UserWarning: Data array used to create a new image contains 64-bit ints. This is likely due to creating the array with numpy and passing `int` as the `dtype`. Many tools such as FSL and SPM cannot deal with int64 in Nifti images, so for compatibility the data has been converted to int32.
  return new_img_like(mask_img, unmasked, affine)


In [29]:
p_map_filename = "./logp_max_t.nii.gz"
out_dict["logp_max_t"].to_filename(p_map_filename)

In [27]:
p_map_filename = "./logp_max_size.nii.gz"
out_dict["logp_max_size"].to_filename(p_map_filename)

In [16]:
z_map = second_level_model.compute_contrast("LSAS", output_type='z_score')

result_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/result/"
temp_dir = f"{seed_anal_dir}/{smoothness}mm/p_map/{group}/{variable}/temp/"
os.makedirs(result_dir, exist_ok=True)
from nilearn.glm import threshold_stats_img
z_map_filename = f"{result_dir}/unthresholded_z_map.nii.gz"
z_map.to_filename(z_map_filename)

thresholded_map2, threshold2 = threshold_stats_img(
    z_map, alpha=0.05, height_control="fdr"
)
print(f"The FDR=.05 threshold is {threshold2:.3g}")
thresholded_map2_filename = f"{result_dir}/FDR_correted_z_map.nii.gz"
thresholded_map2.to_filename(thresholded_map2_filename)

thresholded_map3, threshold3 = threshold_stats_img(
    z_map, alpha=0.05, height_control="bonferroni"
)
print(f"The p<.05 Bonferroni-corrected threshold is {threshold3:.3g}")
thresholded_map3_filename = f"{result_dir}/bonferroni_correted_z_map.nii.gz"
thresholded_map3.to_filename(thresholded_map3_filename)

The FDR=.05 threshold is 2.15
The p<.05 Bonferroni-corrected threshold is 4.71


In [17]:
design_matrix = regressor_df[['SEX', 'AGE', 'YR_EDU', variable, 'Mean_Framewise_Displacement']]

# 인터셉트 열 추가
design_matrix.insert(0, 'intercept', 1)

z_maps = [f"{seed_anal_dir}/{smoothness}mm/corr_z-map/seed_{group}_{variable}/sub-{subject_id}_fisher_z_img.nii.gz" for subject_id in regressor_df['Participant']]

second_level_model = SecondLevelModel(n_jobs=-1)
second_level_model.fit(z_maps, design_matrix=design_matrix)

z_map = second_level_model.compute_contrast(args.variable, output_type='z_score')

result_dir = f"{seed_anal_dir}/{args.smoothness}mm/p_map/{args.group}/{args.variable}/result/"
temp_dir = f"{seed_anal_dir}/{args.smoothness}mm/p_map/{args.group}/{args.variable}/temp/"
os.makedirs(result_dir, exist_ok=True)

p_map_filename = f"{result_dir}/pvalue_map.nii.gz"
p_map.to_filename(p_map_filename)

NameError: name 'args' is not defined